In [ ]:
# Import required libraries
import pandas
import io
from sklearn import tree
from sklearn import svm
import warnings
from sklearn import metrics
from sklearn.metrics import confusion_matrix 

# Suppress warnings to keep output clean
warnings.filterwarnings(action='ignore', category=UserWarning)

# Mount Google Drive to access dataset
from google.colab import drive
drive.mount('/content/gdrive')

# =============================
# 1. Load and Prepare Training Data
# =============================

# Load training data from CSV
df = pandas.read_csv("gdrive/MyDrive/Data /BrisbaneWeatherTrainingData.csv")

# Map categorical fire alert levels to integers
# Original: '-1' = No Alert, 'l' = Low, 'm' = Medium, 'h' = High
# Encoded as: 0 = No Alert, 1 = Low, 2 = Medium, 3 = High
d = {'-1': 0, 'l': 1, 'm': 2, 'h': 3}
df['AlertLevel'] = df['AlertLevel'].map(d)

# Select feature columns (independent variables)
features = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine']
X = df[features]
y = df['AlertLevel']  # Target label

# =============================
# 2. Train Support Vector Regression Model
# =============================

# Create an SVR (Support Vector Regression) model
# ⚠️ NOTE: SVR is meant for predicting continuous values, not classification
regr = svm.SVR()
regr = regr.fit(X, y)

# =============================
# 3. Load and Prepare Test Data
# =============================

# Read the test data
df1 = pandas.read_csv(
    "gdrive/MyDrive/Data /BrisbaneWeatherTestData.csv", 
    usecols=['Date','AlertLevel','MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine']
)

# Map alert levels to numeric format (same as training)
df1['AlertLevel'] = df1['AlertLevel'].map(d)

# =============================
# 4. Make Predictions on Test Data
# =============================

true_y = []  # Ground truth labels
pred_y = []  # Predicted values (continuous)

# Predict for the first 100 rows of test data
for num in range(100):
    true_y.append(df1.iloc[num,1])  # actual value
    pred_y.append(
        regr.predict([[df1.iloc[num,2], df1.iloc[num,3], 
                       df1.iloc[num,4], df1.iloc[num,5], 
                       df1.iloc[num,6]]])[0]
    )

# =============================
# ⚠️ Convert SVR output to classification
# =============================

# Since SVR predicts floating-point values, round them to nearest integer to treat as classification
pred_y = [round(p) for p in pred_y]

# =============================
# 5. Evaluate Model Performance
# =============================

# Calculate accuracy of classification
Accuracy = metrics.accuracy_score(true_y, pred_y)
print("Model Accuracy is: ", Accuracy)

# Print confusion matrix to visualize misclassifications
confusion_matrix = metrics.confusion_matrix(true_y, pred_y)
print("Confusion Matrix is: ")
print(confusion_matrix)

ModuleNotFoundError: No module named 'pandas'